# Docker Image Build & Deploy

In [ ]:
from project_vars import project_variables

env = project_variables['environment']
project_id = project_variables['project_id']
project_name = project_variables['project_name']
region = project_variables['region']

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project {project_id}

In [ ]:
!gcloud auth configure-docker

## Service Build & Deploy

You need to build and deploy as many docker images as you have services. You also need to update services array when services change in `project_vars.py` file. The following code block generates select field that you can use to choose what service you want to deploy.

In [7]:
import ipywidgets as widgets
from IPython.display import display

services = [
    {"name": "auth-service", "folder": "service-auth"},
    {"name": "events-service", "folder": "service-events"},
    {"name": "gateway-service", "folder": "service-gateway"},
    {"name": "storybook-service", "folder": "storybook"},
    {"name": "web-service", "folder": "web"}
]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=[(f"{s['name']} ({s['folder']})", s) for s in services],
    description='Select a service:',
    style={'description_width': 'initial'}
)

# Create output widgets to display the selected values
output_name = widgets.HTML()
output_folder = widgets.HTML()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected = change['new']
        global service_name, service_folder
        service_name = selected['name']
        service_folder = selected['folder']
        output_name.value = f"<b>service_name</b> = \"{service_name}\""
        output_folder.value = f"<b>service_folder</b> = \"{service_folder}\""

# Attach the callback to the dropdown
dropdown.observe(on_change, names='value')

# Display the widgets
display(dropdown, output_name, output_folder)

Dropdown(description='Select a service:', options=(('auth-service (service-auth)', {'name': 'auth-service', 'f…

HTML(value='')

HTML(value='')

In [ ]:
!docker build --no-cache -t {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest -f ../apps/{service_folder}/Dockerfile ../

In [ ]:
!docker push {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest

In [ ]:
!gcloud run deploy {project_id}-{env}-{service_name} --image {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest --platform managed --region={region} --allow-unauthenticated